In [ ]:
# TESSA #1
from google.colab import drive
drive.mount('/content/drive') # for google colab. adjust accordingly
PARENT_DIR = '/content/drive/MyDrive/CS287-final-project/' # for google colab. adjust accordingly

KeyboardInterrupt: ignored

In [ ]:
# #check gpu connection
# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)

Tue Nov 23 19:25:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# TESSA #2
!pip install gpytorch

     |████████████████████████████████| 503 kB 8.5 MB/s 


In [ ]:
# TESSA #3
!git clone https://github.com/Roxot/mbr-nmt.git
%cd mbr-nmt
!pip install .

Cloning into 'mbr-nmt'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 313 (delta 1), reused 1 (delta 0), pack-reused 306
Receiving objects: 100% (313/313), 206.35 MiB | 20.94 MiB/s, done.
Resolving deltas: 100% (186/186), done.
/content/mbr-nmt
Processing /content/mbr-nmt
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 895 kB 13.1 MB/s 
     |████████████████████████████████| 78 kB 10.6 MB/s 
  Created wheel for mbr-nmt: filename=mbr_nmt-0.1.dev0-py3-none-any.whl size=24919 sha256=25f8241adc3a29069a4e71097

In [ ]:
# function to format list of candidates as required by MBR translate function
def convert_to_candidate_file(unformatted_filename, formatted_filename, n_candidates):
  with open(unformatted_filename, 'r') as unformatted, open(formatted_filename, 'w+') as formatted:
    n_lines = len(unformatted.readlines())
    n_sentences = int(n_lines / n_candidates)
    unformatted.seek(0)
    for i in range(n_sentences):
      formatted.write('NC=%d\n' % n_candidates)
      for c in range(n_candidates):
        line = unformatted.readline()
        formatted.write(line)

In [ ]:
# save candidate files in format requried by MBR translate function
convert_to_candidate_file(PARENT_DIR + 'topk_translations/candidates_beam_beam5.txt', PARENT_DIR + 'topk_translations/candidates_beam_beam5_c.txt', 5)
convert_to_candidate_file(PARENT_DIR + 'topk_translations/candidates_topk_k10_n400.txt', PARENT_DIR + 'topk_translations/candidates_topk_k10_n400_c.txt', 400)
convert_to_candidate_file(PARENT_DIR + 'topk_translations/basic_candidates_topp_p70_n400.txt', PARENT_DIR + 'topk_translations/basic_candidates_topp_p70_n400_c.txt', 400)

In [ ]:
convert_to_candidate_file(PARENT_DIR + 'topk_translations/candidates_topp_p90_n400.txt', PARENT_DIR + 'topk_translations/candidates_topp_p90_n400_c.txt', 400)

In [ ]:
convert_to_candidate_file(PARENT_DIR + 'topk_translations/basic_candidates_ancestral_n1000.txt', PARENT_DIR + 'topk_translations/basic_candidates_ancestral_n1000_c.txt', 1000)

In [ ]:
for beam_size in range(1,11):
  convert_to_candidate_file(PARENT_DIR + f'topk_translations/candidates_beam_beam{beam_size}.txt', PARENT_DIR + f'topk_translations/candidates_beam_beam{beam_size}_c.txt', beam_size)

In [ ]:
for k in [10,20,30,40,50,60,70,80,90,100]:
  convert_to_candidate_file(PARENT_DIR + f'topk_translations/candidates_topk_k{k}_n400.txt', PARENT_DIR + f'topk_translations/candidates_topk_k{k}_n400_c.txt', 400)

In [ ]:
for p in [x * 10 for x in range(1, 11)]:
  convert_to_candidate_file(PARENT_DIR + f'topk_translations/candidates_topp_p{p}_n400.txt', PARENT_DIR + f'topk_translations/candidates_topp_p{p}_n400_c.txt', 400)

In [ ]:
# TESSA #4
# translation via MBR decoding
def mbr_from_sample(sample_output_filename, candidate_output_filename, final_output_filename, c2f, 
                    num_samples, subsample_size = 100, subsample_size_2 = 200, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu'):
  # create outMBR.txt
  if c2f:
    !mbr-nmt translate -s $sample_output_filename -n {num_samples} -u {utility} -o outMBR.txt -c $candidate_output_filename --c2f -mc1 {subsample_size} -mc2 {subsample_size_2} --top-k {top_k} -u2 {utility_2}
  else:
    !mbr-nmt translate -s $sample_output_filename -n {num_samples} -u {utility} -o outMBR.txt -c $candidate_output_filename
  # outMBR.txt --> final_output_filename
  !mbr-nmt convert -f mbr-nmt -i outMBR.txt -o $final_output_filename

In [ ]:
# candidates = ancestral with n = 1000
# samples = 200 ancestral samples
mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n200.txt', 
                PARENT_DIR + 'topk_translations/basic_candidates_ancestral_n1000_c.txt', 
                PARENT_DIR + 'MBR/samples_ancestral_n200_basic_candidates_ancestral_n1000_translations.txt', 
                c2f = True, num_samples = 200, subsample_size = 100, subsample_size_2 = 200, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=True, candidates='/content/drive/MyDrive/CS287-final-project/topk_translations/basic_candidates_ancestral_n1000_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=200, output_file='outMBR.txt', samples='/content/drive/MyDrive/CS287-final-project/topk_translations/samples_ancestral_n200.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=100, subsample_size_2=200, threads=-1, top_k=50, utility='skip-bigram-f1', utility_2='bleu')
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.
Decoding took 9165s


In [ ]:
# # candidates = beam search with beam size = 5
# # samples = 100 ancestral samples
# # because the number of candidates is already less than 50, C2F doesn't actually narrow the hypothesis space so might not make sense in this context
# mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n100.txt', 
#                 PARENT_DIR + 'topk_translations/candidates_beam_beam5_c.txt', 
#                 PARENT_DIR + 'MBR/beam_beam5_translations.txt', c2f = True, num_samples = 100, subsample_size = 100, subsample_size_2 = 100, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=True, candidates='/content/drive/MyDrive/CS287-final-project/topk_translations/candidates_beam_beam5_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=100, output_file='outMBR.txt', samples='/content/drive/MyDrive/CS287-final-project/topk_translations/samples_ancestral_n100.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=100, subsample_size_2=100, threads=-1, top_k=50, utility='skip-bigram-f1', utility_2='bleu')
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.
Decoding took 418s


In [ ]:
# candidates = beam search with beam size = 5
# samples = 200 ancestral samples
mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n200.txt', 
                PARENT_DIR + 'topk_translations/basic_candidates_beam_beam5_c.txt', 
                PARENT_DIR + 'MBR/samples_ancestral_n200_basic_candidates_beam_beam5_translations.txt', 
                c2f = True, num_samples = 200, subsample_size = 100, subsample_size_2 = 200, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=True, candidates='/content/drive/MyDrive/CS287-final-project/topk_translations/basic_candidates_beam_beam5_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=200, output_file='outMBR.txt', samples='/content/drive/MyDrive/CS287-final-project/topk_translations/samples_ancestral_n200.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=100, subsample_size_2=200, threads=-1, top_k=50, utility='skip-bigram-f1', utility_2='bleu')
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.
Decoding took 796s


In [ ]:
# candidates = beam search with varying beam size
# samples = 200 ancestral samples
for beam_size in range(1,11):
  mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n200.txt', 
                  PARENT_DIR + f'topk_translations/candidates_beam_beam{beam_size}_c.txt', 
                  PARENT_DIR + f'MBR/samples_ancestral_n200_candidates_beam_beam{beam_size}_translations.txt', 
                  c2f = True, num_samples = 200, subsample_size = 100, subsample_size_2 = 200, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=True, candidates='/content/drive/MyDrive/CS287-final-project/topk_translations/candidates_beam_beam10_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=200, output_file='outMBR.txt', samples='/content/drive/MyDrive/CS287-final-project/topk_translations/samples_ancestral_n200.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=100, subsample_size_2=200, threads=-1, top_k=50, utility='skip-bigram-f1', utility_2='bleu')
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.
Decoding took 1469s


In [ ]:
# # candidates = top k with k = 10, n = 400
# # samples = 100 ancestral samples
# mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n100.txt', 
#                 PARENT_DIR + 'topk_translations/candidates_topk_k10_n400_c.txt', 
#                 PARENT_DIR + 'MBR/basic_candidates_topk_k10_n400_translations.txt', c2f = True, num_samples = 100, subsample_size = 100, subsample_size_2 = 100, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=True, candidates='/content/drive/MyDrive/CS287-final-project/topk_translations/candidates_topk_k10_n400_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=100, output_file='outMBR.txt', samples='/content/drive/MyDrive/CS287-final-project/topk_translations/samples_ancestral_n100.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=100, subsample_size_2=100, threads=-1, top_k=50, utility='skip-bigram-f1', utility_2='bleu')
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.
Decoding took 4420s


In [ ]:
# candidates = top k with k = 10, n = 400
# samples = 200 ancestral samples
mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n200.txt', 
                PARENT_DIR + 'topk_translations/basic_candidates_topk_k10_n400_c.txt', 
                PARENT_DIR + 'MBR/samples_ancestral_n200_basic_candidates_topk_k10_n400_translations.txt', 
                c2f = True, num_samples = 200, subsample_size = 100, subsample_size_2 = 200, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=True, candidates='/content/drive/MyDrive/CS287-final-project/topk_translations/basic_candidates_topk_k10_n400_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=200, output_file='outMBR.txt', samples='/content/drive/MyDrive/CS287-final-project/topk_translations/samples_ancestral_n200.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=100, subsample_size_2=200, threads=-1, top_k=50, utility='skip-bigram-f1', utility_2='bleu')
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.
Decoding took 8118s


In [ ]:
PARENT_DIR = '/content/drive/MyDrive/cs287/CS287-final-project/' # for google colab. adjust accordingly
# TESSA #5
# candidates = top k with varying k, n = 400
# samples = 200 ancestral samples
#for k in [20,50,100]:
for k in [10,30,40,60,70,80,90]:
  mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n200.txt', 
                  PARENT_DIR + f'topk_translations/candidates_topk_k{k}_n400_c.txt', 
                  PARENT_DIR + f'MBR/samples_ancestral_n200_candidates_topk_k{k}_n400_translations.txt', 
                  c2f = True, num_samples = 200, subsample_size = 100, subsample_size_2 = 200, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=True, candidates='/content/drive/MyDrive/cs287/CS287-final-project/topk_translations/candidates_topk_k10_n400_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=200, output_file='outMBR.txt', samples='/content/drive/MyDrive/cs287/CS287-final-project/topk_translations/samples_ancestral_n200.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=100, subsample_size_2=200, threads=-1, top_k=50, utility='skip-bigram-f1', utility_2='bleu')
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.


In [ ]:
# # candidates = top p with p = 70, n = 400
# # samples = 100 ancestral samples
# mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n100.txt', 
#                 PARENT_DIR + 'topk_translations/basic_candidates_topp_p70_n400_c.txt', 
#                 PARENT_DIR + 'MBR/basic_candidates_topp_p70_n400_translations.txt', c2f = True, num_samples = 100, subsample_size = 100, subsample_size_2 = 100, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=True, candidates='/content/drive/MyDrive/CS287-final-project/topk_translations/basic_candidates_topp_p70_n400_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=100, output_file='outMBR.txt', samples='/content/drive/MyDrive/CS287-final-project/topk_translations/samples_ancestral_n100.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=100, subsample_size_2=100, threads=-1, top_k=50, utility='skip-bigram-f1', utility_2='bleu')
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.
Decoding took 4714s


In [ ]:
# candidates = top p with p = 70, n = 400
# samples = 200 ancestral samples
mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n200.txt', 
                PARENT_DIR + 'topk_translations/basic_candidates_topp_p70_n400_c.txt', 
                PARENT_DIR + 'MBR/samples_ancestral_n200_basic_candidates_topp_p70_n400_translations.txt', c2f = True, num_samples = 200, subsample_size = 100, subsample_size_2 = 200, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=True, candidates='/content/drive/MyDrive/CS287-final-project/topk_translations/basic_candidates_topp_p70_n400_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=200, output_file='outMBR.txt', samples='/content/drive/MyDrive/CS287-final-project/topk_translations/samples_ancestral_n200.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=100, subsample_size_2=200, threads=-1, top_k=50, utility='skip-bigram-f1', utility_2='bleu')
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.


In [ ]:
# ALAN #5
# candidates = top p with varying p, n = 400
# samples = 200 ancestral samples
ps =  [x * 10 for x in range(1, 11)]
ps.remove(70)
ps.reverse()
for p in ps:
  mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n200.txt', 
                  PARENT_DIR + f'topk_translations/candidates_topp_p{p}_n400_c.txt', 
                  PARENT_DIR + f'MBR/samples_ancestral_n200_candidates_topp_p{p}_n400_translations.txt', 
                  c2f = True, num_samples = 200, subsample_size = 100, subsample_size_2 = 200, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=True, candidates='/content/drive/MyDrive/CS287-final-project/topk_translations/candidates_topp_p100_n400_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=200, output_file='outMBR.txt', samples='/content/drive/MyDrive/CS287-final-project/topk_translations/samples_ancestral_n200.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=100, subsample_size_2=200, threads=-1, top_k=50, utility='skip-bigram-f1', utility_2='bleu')
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.


In [ ]:
# candidates = top p with p = 90, n = 400
# samples = 200 ancestral samples
mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n200.txt', 
                PARENT_DIR + 'topk_translations/candidates_topp_p90_n400_c.txt', 
                PARENT_DIR + 'MBR/samples_ancestral_n200_candidates_topp_p90_n400_translations.txt', 
                c2f = True, num_samples = 200, subsample_size = 100, subsample_size_2 = 200, top_k = 50, utility = 'skip-bigram-f1', utility_2 = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=True, candidates='/content/drive/MyDrive/CS287-final-project/topk_translations/candidates_topp_p90_n400_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=200, output_file='outMBR.txt', samples='/content/drive/MyDrive/CS287-final-project/topk_translations/samples_ancestral_n200.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=100, subsample_size_2=200, threads=-1, top_k=50, utility='skip-bigram-f1', utility_2='bleu')
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.


In [ ]:
# same as above but without C2F to compare computation time
# candidates = top p with p = 70, n = 400
# samples = 200 ancestral samples
mbr_from_sample(PARENT_DIR + 'topk_translations/samples_ancestral_n200.txt', 
                PARENT_DIR + 'topk_translations/basic_candidates_topp_p70_n400_c.txt', 
                PARENT_DIR + 'MBR/samples_ancestral_n200_basic_candidates_topp_p70_n400_translations_noC2F.txt', c2f = False,
                num_samples = 200, utility = 'bleu')

Namespace(add_eos=False, bleurt_checkpoint=None, bmc=False, c2f=False, candidates='/content/drive/MyDrive/CS287-final-project/topk_translations/basic_candidates_topp_p70_n400_c.txt', command='translate', kernel_utility=None, lang='en', num_samples=200, output_file='outMBR.txt', samples='/content/drive/MyDrive/CS287-final-project/topk_translations/samples_ancestral_n200.txt', seed=None, store_expected_utility=None, subsample_per_candidate=False, subsample_size=None, subsample_size_2=None, threads=-1, top_k=None, utility='bleu', utility_2=None)
Starting 2 parallel processes.
Started process 1 for doing 1250 translations.
Started process 2 for doing 1250 translations.
